# For Rounding glyphs & Metric Adjustments

In [ ]:
%reset
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
import numpy as np
from scipy.optimize import fsolve, fmin
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [ ]:
# Common Func    
def Strp2cod():
    global cod,cmd,typ,xmin,xmax,ymin,ymax
    cod=[]
    cmd=[]
    typ=[]
    codsts=0
    codtmp=0.0
    lth=len(strp)
    i_s=i_e=0
    while (i_e<lth):
        i_s=i_e
        while (strp[i_e]!=" " and strp[i_e]!="\n"):
            i_e+=1
        if(strp[i_s]=="m"): #move to
            codsts=3
            cmd=cmd+[Path.MOVETO]
        elif(strp[i_s]=="l"): #line to
            codsts=3
            cmd=cmd+[Path.LINETO]
        elif(strp[i_s]=="c"): #curve to
            codsts=3
            cmd=cmd+[Path.CURVE4, Path.CURVE4, Path.CURVE4]
        elif(codsts==0): #not pairing
            codtmp=float(strp[i_s:i_e])
            codsts=1
        elif(codsts==1): #pairing
            cod=cod+[np.array([codtmp,float(strp[i_s:i_e])])]
            codsts=0
        elif(codsts==3): #add type
            codsts=0
            typ=typ+[strp[i_s:i_e]]
        while (i_e<lth and (strp[i_e]==" " or strp[i_e]=="\n")):
            i_e+=1
    xmin=10000
    xmax=-10000
    ymin=10000
    ymax=-10000
    for i_ in range(len(cmd)):
        if(cod[i_][0]<xmin):
            xmin=cod[i_][0]
        elif(cod[i_][0]>xmax):
            xmax=cod[i_][0]
        if(cod[i_][1]<ymin):
            ymin=cod[i_][1]
        elif(cod[i_][1]>ymax):
            ymax=cod[i_][1]
            
def PlotGlyph(grd):
    path = Path(cod, cmd)
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    patch = patches.PathPatch(path, facecolor='black', lw=2)
    ax.add_patch(patch)
    plt.ylim(ymin,ymax)
    plt.xlim(xmin,xmax)
    ax.set_aspect('equal')
    if(grd):
        ax.set_xticks(np.arange(xmin,xmax,50), minor=True)
        ax.set_yticks(np.arange(ymin,ymax,50), minor=True)
        ax.grid(which='both')
    plt.show()
    
def Cod2cstr():
    global cstr
    cstr=''
    j_=0
    for i_ in range(len(typ)):
        if(cmd[j_]==Path.MOVETO):
            cstr=cstr+str(cod[j_][0])+' '+str(cod[j_][1])+' m '+typ[i_]+'\n'
            j_+=1
        elif(cmd[j_]==Path.LINETO):
            cstr=cstr+' '+str(cod[j_][0])+' '+str(cod[j_][1])+' l '+typ[i_]+'\n'
            j_+=1
        elif(cmd[j_]==Path.CURVE4):
            for ii_ in range(3):
                cstr=cstr+' '+str(cod[j_][0])+' '+str(cod[j_][1])
                j_+=1
            cstr=cstr+' c '+typ[i_]+'\n'
    #print(cstr)
    

In [ ]:
# Round

def ReadGlyph(g_fname): #extract to strp
    global strp
    hfile = open(g_fname)
    lnstr=hfile.readline()
    #Move to SplineSet
    while (lnstr[:9]!="SplineSet"):
        lnstr=hfile.readline()

    lnstr=hfile.readline()
    strp=''
    while (lnstr[:12]!="EndSplineSet"):
        strp=strp+lnstr
        lnstr=hfile.readline()
    hfile.close()

X0,X1,X2,X3=7,2,3,5
Y0,Y1,Y2,Y3=7,8,4,0

def X(t_):
    return (1-t_)**3*X0+3*t_*(1-t_)**2*X1+3*t_**2*(1-t_)*X2+t_**3*X3

def Y(t_):
    return (1-t_)**3*Y0+3*t_*(1-t_)**2*Y1+3*t_**2*(1-t_)*Y2+t_**3*Y3

def NX(NX1_,NX2_,u_,t_):
    return (1-t_)**3*X(u_)+3*t_*(1-t_)**2*NX1_+3*t_**2*(1-t_)*NX2_+t_**3*X3

def NY(NY1_,NY2_,u_,t_):
    return (1-t_)**3*Y(u_)+3*t_*(1-t_)**2*NY1_+3*t_**2*(1-t_)*NY2_+t_**3*Y3

def Eqn4GN(pm_,z_):
    NX1_,NY1_,NX2_,NY2_=pm_
    return (NX(NX1_,NX2_,z_,0.3)-X(z_ + 0.3*(1 - z_)),NY(NY1_,NY2_,z_,0.3)-Y(z_ + 0.3*(1 - z_)),
            NX(NX1_,NX2_,z_,0.7)-X(z_ + 0.7*(1 - z_)),NY(NY1_,NY2_,z_,0.7)-Y(z_ + 0.7*(1 - z_)))

def GN(z_): # cut Bezier curve @ z
    NX1,NY1,NX2,NY2=(fsolve(Eqn4GN,(X1,Y1,X2,Y2),z_))
    return [(X(z_),Y(z_)),(NX1,NY1),(NX2,NY2),(X3,Y3)]

def Crs(vc1_, vc2_):
    xx1_,yy1_=vc1_
    xx2_,yy2_=vc2_
    nm_=np.sqrt(xx1_**2+yy1_**2)*np.sqrt(xx2_**2+yy2_**2)
    return (xx1_*yy2_-xx2_*yy1_)/nm_

def Func4CC(lmd_,tgx_,tgy_):
    return (X(lmd_)-tgx_)**2+(Y(lmd_)-tgy_)**2

def CutCV(sttpt_,cvdir_,ctlth_):
        global X0,X1,X2,X3,Y0,Y1,Y2,Y3
        X0,Y0=cod[sttpt_]
        if(cvdir_>0):
            X1,Y1=cod[sttpt_+1]
            X2,Y2=cod[sttpt_+2]
            X3,Y3=cod[sttpt_+3]
        else:
            X1,Y1=cod[sttpt_-1]
            X2,Y2=cod[sttpt_-2]
            X3,Y3=cod[sttpt_-3]
        dr_=np.array([X1,Y1])-np.array([X0,Y0])
        dr_=np.array([X0,Y0])+dr_/np.sqrt(dr_[0]**2+dr_[1]**2)*ctlth_
        return (fmin(Func4CC,0,(dr_[0],dr_[1]),disp=0))[0]
    
def RdCurve():
    global chkpt,prvcb,lth,typidx
    if(cmd[chkpt]==Path.CURVE4): #first curve is bz4
        λ=CutCV(chkpt-1,1,ctlth)
        cgrst1=GN(λ)
    elif(cmd[chkpt]==Path.LINETO): #line
        cgrst1=np.array(cod[chkpt])-np.array(cod[chkpt-1])
        cgrst1=[np.array(cod[chkpt-1])+cgrst1/np.sqrt(cgrst1[0]**2+cgrst1[1]**2)*ctlth]
    else: print('Error@chkpt: ',chkpt)
    
    if(cmd[prvcb]==Path.CURVE4): #last curve is bz4
        λ=CutCV(prvcb,-1,ctlth)
        cgrst2=GN(λ)
    elif(cmd[prvcb]==Path.LINETO): #line
        cgrst2=np.array(cod[prvcb-1])-np.array(cod[prvcb])
        cgrst2=[np.array(cod[prvcb])+cgrst2/np.sqrt(cgrst2[0]**2+cgrst2[1]**2)*ctlth]
    else: print('Error@prvcb: ',prvcb)
        
    #assign type
    if(cmd[prvcb]==Path.CURVE4):
        typ[typidx-1]=typ[tpidpv]='4'
    else:
        typ[typidx-1]=typ[tpidpv]='2'
    
    if(cmd[chkpt]==Path.CURVE4):
        typ.insert(typidx,'4')
    else:
        typ.insert(typidx,'2')
    
    #assign last cuv
    if(cmd[prvcb]==Path.CURVE4): #last curve is bz4
        cod[chkpt-1]=cod[prvcb]=cgrst2[0] #this deals the starting point automatically
        cod[prvcb-1]=cgrst2[1]
        cod[prvcb-2]=cgrst2[2]
    else:
        cod[chkpt-1]=cod[prvcb]=cgrst2[0] 
        
    #assign first cuv
    if(cmd[chkpt]==Path.CURVE4): #last curve is bz4
        cod[chkpt]=cgrst1[1]
        cod[chkpt+1]=cgrst1[2] 
        
    #assign inserted cuv
    cod.insert(chkpt,cgrst1[0])
    cmd.insert(chkpt,Path.CURVE4)
    
    dr=np.array(cod[chkpt])-np.array(cod[chkpt+1])
    dr=np.array(cod[chkpt])+dr/np.sqrt(dr[0]**2+dr[1]**2)*ctlth*0.666666666666666666666666666666666667
    cod.insert(chkpt,dr)
    cmd.insert(chkpt,Path.CURVE4)
    
    if(prvcb>chkpt): prvcb+=2
    dr=np.array(cod[prvcb])-np.array(cod[prvcb-1])
    dr=np.array(cod[prvcb])+dr/np.sqrt(dr[0]**2+dr[1]**2)*ctlth*0.666666666666666666666666666666666667
    cod.insert(chkpt,dr)
    cmd.insert(chkpt,Path.CURVE4)
    chkpt+=3
    lth+=3
    typidx+=1

def Round():
    global ctlth
    tmp=Crs(np.array(cod[chkpt])-np.array(cod[prvcb]),
            np.array(cod[prvcb-1])-np.array(cod[prvcb]))
    if(tmp>.1): ctlth=10.0 #outside angle
    elif(tmp<-.1): ctlth=30.0 #inner angle
    else: return 0 #no need to round
    RdCurve()
    return 1

def WriteGlyph(g_fname): #write cstr to file
    hfile = open(g_fname)
    lpstr=opr=hfile.readline()
    while (lpstr[:9]!="SplineSet"):
        lpstr=hfile.readline()
        opr+=lpstr
        
    lpstr=hfile.readline()
    while (lpstr[:12]!="EndSplineSet"):
        lpstr=hfile.readline()
    
    opr=opr+cstr+lpstr+hfile.read()
    hfile.close()
    
    hfile=open(g_fname,"w")
    hfile.write(opr)
    hfile.close

def RoundGlyph():
    global chkpt,prvcb,typidx,tpidpv,lth
    cbase=0 #point to the beginning 'MOVETO'
    chkpt=0 #current check point: beginning of the drawing command
    prvcb=0 #last curve end pos
    tbase=0
    typidx=0
    tpidpv=0
    ctlth=0
    lth=len(cod)
    while (True):
        if(chkpt>=lth or cmd[chkpt]==Path.MOVETO):
            #deal prev.
            if(chkpt>0):
                tppos=chkpt
                tpypp=typidx
                chkpt=cbase
                typidx=tbase
                dn=Round()
                chkpt=tppos+3*dn
                typidx=tpypp+dn
            if(chkpt>=lth): break

            cbase=chkpt+1
            if(cmd[chkpt+1]==Path.LINETO): chkpt+=2
            else: chkpt+=4
            prvcb=chkpt-1

            tbase=typidx+1
            typidx+=2
            tpidpv=typidx-1

        elif(cmd[chkpt]==Path.LINETO):
            Round()
            chkpt+=1
            prvcb=chkpt-1
            typidx+=1
            tpidpv=typidx-1

        elif(cmd[chkpt]==Path.CURVE4):
            Round()
            chkpt+=3
            prvcb=chkpt-1
            typidx+=1
            tpidpv=typidx-1

        else:
            print('UnknownCommand@chkpt: ',chkpt)
            

In [ ]:
import os
folder='../../gtemp/Untitled1.sfdir/'
for path, subdirs, files in os.walk(folder):
    for filename in files:
        if(filename != 'font.props'):
            print (filename, '...')
            ReadGlyph(folder+filename)
            Strp2cod()
            RoundGlyph()
            Cod2cstr()
            WriteGlyph(folder+filename)
            print ('Done')
print("finished")            

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def ReadGlfWdIc(g_fname):
    global strp
    hfile = open(g_fname)
    lnstr="begin"
    #Get Width, ItalicCor, TopAccentPos
    wd=itacor=tpach=0 # init, in case of no info provided

    while (lnstr[:4]!="Fore"):
        lnstr=hfile.readline()
        if (lnstr[:5]=="Width"):
            wd=float(lnstr[7:])
            continue
        if (lnstr[:16]=="ItalicCorrection"):
            itacor=float(lnstr[18:])
            continue
        if (lnstr[:19]=="TopAccentHorizontal"):
            tpach=float(lnstr[21:])
            continue

    #Move to SplineSet
    while (lnstr[:9]!="SplineSet"):
        lnstr=hfile.readline()

    lnstr=hfile.readline()
    strp=''
    while (lnstr[:12]!="EndSplineSet"):
        strp=strp+lnstr
        lnstr=hfile.readline()
    hfile.close()
    return (wd,itacor,tpach)

In [ ]:
#Adjust left
wd1,itacor1,tpach1=ReadGlfWdIc('../Garamond-Math.sfdir/_Q.glyph')
Strp2cod()
cod1,cmd1,typ1=np.array(cod),cmd,typ
xmin1,xmax1,ymin1,ymax1=xmin,xmax,ymin,ymax

wd2,itacor2,tpach2=ReadGlfWdIc('../Garamond-Math.sfdir/_A.glyph')
Strp2cod()
cod2,cmd2,typ2=np.array(cod),cmd,typ
xmin2,xmax2,ymin2,ymax2=xmin,xmax,ymin,ymax

cod2+=np.array([wd1+itacor1,0])
xmin2+=(wd1+itacor1)
xmax2+=(wd1+itacor1)

cmd=cmd1+cmd2

xmin, xmax = 0, 2000
ymin=-300
ymax=700

def PltGlfI(dx,grd):
    global cod
    cod=np.concatenate((cod1,np.array(cod2)+np.array([dx,0])))
    PlotGlyph(grd)

interact(PltGlfI, dx=(-200,200,10),grd=True)


In [ ]:
#Read Mathtable Const
hfile = open("../Garamond-Math.sfdir/font.props")
lnstr="begin"
while (lnstr!=""):
    lnstr=hfile.readline()
    if (lnstr[5:27]=="ScriptPercentScaleDown"):
        ScriptPercentScaleDown=int(lnstr[29:])
    elif (lnstr[5:33]=="ScriptScriptPercentScaleDown"):
        ScriptScriptPercentScaleDown=int(lnstr[35:])
    elif (lnstr[5:23]=="SubscriptShiftDown"):
        SubscriptShiftDown=int(lnstr[25:])
    elif (lnstr[5:20]=="SubscriptTopMax"):
        SubscriptTopMax=int(lnstr[22:])
    elif (lnstr[5:29]=="SubscriptBaselineDropMin"):
        SubscriptBaselineDropMin=int(lnstr[31:])
    elif (lnstr[5:24]=="SuperscriptShiftUp:"):
        SuperscriptShiftUp=int(lnstr[25:])
    elif (lnstr[5:25]=="SuperscriptBottomMin"):
        SuperscriptBottomMin=int(lnstr[27:])
    elif (lnstr[5:31]=="SuperscriptBaselineDropMax"):
        SuperscriptBaselineDropMax=int(lnstr[33:])
        
print(ScriptPercentScaleDown,
      ScriptScriptPercentScaleDown,
      SubscriptShiftDown,
      SubscriptTopMax,
      SubscriptBaselineDropMin,
      SuperscriptShiftUp,
      SuperscriptBottomMin,
      SuperscriptBaselineDropMax
    )

In [ ]:
#Adjust left
wd1,itacor1,tpach1=ReadGlfWdIc('../Garamond-Math.sfdir/u1D449.glyph')
Strp2cod()
cod1,cmd1,typ1=np.array(cod),cmd,typ
xmin1,xmax1,ymin1,ymax1=xmin,xmax,ymin,ymax

wd2,itacor2,tpach2=ReadGlfWdIc('../Garamond-Math.sfdir/u1D465.glyph')
Strp2cod()
cod2,cmd2,typ2=np.array(cod)*(ScriptPercentScaleDown/100.0),cmd,typ

xmin2,xmax2,ymin2,ymax2=np.array([xmin,xmax,ymin,ymax])*(ScriptPercentScaleDown/100.0)

cmd=cmd1+cmd2+cmd2
xmin, xmax = 0, 2000
ymin=-300
ymax=900
for ixx in np.arange(-400,1000,50):
    cmd+=[Path.MOVETO,Path.LINETO]



def PltGlfIS(dw,di,extd,grd):
    global cod,cmd
    infShift=-SubscriptShiftDown
    if(ymax2-SubscriptShiftDown>SubscriptTopMax):
        infShift=SubscriptTopMax-max2
    if(extd==True and ymin1-SubscriptBaselineDropMin<infShift):
        infShift=ymin1-SubscriptBaselineDropMin
        
    supShift=SuperscriptShiftUp
    if(ymin2+SuperscriptShiftUp<SuperscriptBottomMin):
        supShift=SuperscriptBottomMin-min2
    if(extd==True and ymax1-SuperscriptBaselineDropMax>supShift):
        supShift=ymax1-SuperscriptBaselineDropMax       
    
    cod=np.concatenate((cod1,
                        np.array(cod2)+np.array([wd1+dw,infShift]),
                        np.array(cod2)+np.array([wd1+dw+itacor1+di,supShift])))
    
    islope=(itacor1+di)/(supShift-infShift) 
    for ixx in np.arange(-400,1000,50):
        cod=np.concatenate((cod, np.array([[ixx,ymin],[ixx+(ymax-ymin)*islope,ymax]])))
    
    PlotGlyph(grd)

interact(PltGlfIS, dw=(-200,200,10),di=(-200,200,10),extd=False,grd=False)

In [ ]:
# z=int("AA",base=16)
# hex(z).upper()[2:]

#Warning will write from data **1
def WriteGlyphIc():
    return 0;